In [ ]:
!pip install beautifulsoup4

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from difflib import SequenceMatcher
pd.set_option('display.max_rows', 1000000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
q = 'engenheiro de dados'
response = requests.get(f'https://br.linkedin.com/jobs/search?keywords={q}&location=Brasil&locationId=&geoId=106057199&f_TPR=r86400&position=1&pageNum=0', verify=False)
soup = bs(response.text, 'html.parser')


c:\Users\XQ6460\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'br.linkedin.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [94]:
import warnings
warnings.filterwarnings('ignore')

In [99]:
def get_jobs_links(q):

    #faz a primeira pesquisa para captar o número de páginas resultantes da pesquisa
    response = requests.get(f'https://www.linkedin.com/jobs/search/?currentJobId=3640009990&f_TPR=r86400&geoId=106057199&keywords={q}&location=Brasil&originalSubdomain=br', verify=False)
    soup = bs(response.text, 'html.parser')

    # recebe o número de resultados da pesquisa e cria uma lista com os parametros necessários para acessar todas as páginas.
    try:
        num_of_results = soup.find(class_ = 'results-context-header__job-count').text
        num_of_pages = int(int(num_of_results)/25)
        range_num_of_pages = list(range(0,num_of_pages))
        start_key_list = [x *25 for x in range_num_of_pages]
        start_key_list
    except ValueError:
        start_key_list = [0,25,50,75,100,125,150,175,200,225]
    
    links = {}
    #acessa cada uma das páginas do resultado da pesquisa
    for key in start_key_list:
        response = requests.get(f'https://www.linkedin.com/jobs/search/?currentJobId=3640009990&f_TPR=r86400&geoId=106057199&keywords={q}&location=Brasil&originalSubdomain=br&start={key}', verify=False)
        soup = bs(response.text, 'html.parser')
        items = soup.findAll(class_='base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]')
        
        
        #verifica se o título do item corresponde a pesquisa
        for item in items:
            job_title = str(item.span.text).strip()
            s = SequenceMatcher(None, q, job_title)
            if s.ratio() > 0.6:
                links[job_title] = item.get('href')
            
    return links 

    


In [110]:
list_of_links = get_jobs_links('analista de dados')

lista0
https://br.linkedin.com/jobs/view/analista-de-dados-remoto-at-doc9-3642259767?refId=j3pQv6jkLI9dPkLROjZmjA%3D%3D&trackingId=lR7uYEzLlDvvCuqnXtPAMA%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card
https://br.linkedin.com/jobs/view/analista-de-bi-j%C3%BAnior-at-mark-up-3641183593?refId=j3pQv6jkLI9dPkLROjZmjA%3D%3D&trackingId=CHOUjnPsKj1xKUcOxFKicg%3D%3D&position=2&pageNum=0&trk=public_jobs_jserp-result_search-card
https://br.linkedin.com/jobs/view/analista-de-dados-produtos-financeiros-at-santander-brasil-3641159828?refId=j3pQv6jkLI9dPkLROjZmjA%3D%3D&trackingId=PR3eTtCuaSUoNjBZD0FVVg%3D%3D&position=3&pageNum=0&trk=public_jobs_jserp-result_search-card
https://br.linkedin.com/jobs/view/business-intelligence-analyst-at-keyrus-3630381761?refId=j3pQv6jkLI9dPkLROjZmjA%3D%3D&trackingId=uzGuT4JDjU3jBryq%2BR0p7Q%3D%3D&position=4&pageNum=0&trk=public_jobs_jserp-result_search-card
https://br.linkedin.com/jobs/view/analista-de-dados-at-grupo-rfk-3642091995?refId=j3pQv6jkLI9

In [83]:
"""
list_of_links = []
for item in soup.findAll(class_='base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]'):
    list_of_links.append(item.get('href'))
len(list_of_links)
#armazenar no banco de dados a lista de links
#armazenar no banco de dados o arquivo html
"""

22

11

In [118]:
list_of_prequisites = []
for link in list(list_of_links.values()):
    response = requests.get(link, verify=False)
    soup = bs(response.text, 'html.parser')
    try:
        list_of_prequisites.append(soup.find(class_='description__text description__text--rich').find_all('li'))
    except(AttributeError):
        pass
len(list_of_prequisites)

10

In [9]:
#cria uma pasta para a query
#armazenar list_of_prequisites junto com timestamp (nosql)
#armazenar lista de links junto com timestamp (sql)
#recuperar lista de links e tranforma em texto apenas os links que não estiverem na lista
#append lista de links junto com timestamp
#importa todos os textos que estão marcados como a query especificada

list

In [ ]:
"""dict_of_prequisites = {}
for link in list_of_links:
    response = requests.get(link, verify=False)
    soup = bs(response.text, 'html.parser')
    try:
        dict_of_prequisites[link] = (soup.find(class_='description__text description__text--rich').find_all('li'))
    except(AttributeError):
        pass
len(dict_of_prequisites)"""


In [5]:
text = str(list_of_prequisites).replace('<li>','').replace('</li>','')\
    .replace('</li>',"")\
    .replace('<li>',"")\
    .replace('[',"")\
    .replace(']',"")\
    .replace('<br/>',"")\
    .replace('<br>',"")\
    .replace('</span>',"")\
    .replace('<span>',"")\
    .replace('.',"")\
    .replace(',',"")\
    .replace(';',"")\
    .lower()
    
words = str(list_of_prequisites).replace('<li>','').replace('</li>','')\
    .replace('</li>',"")\
    .replace('<li>',"")\
    .replace('[',"")\
    .replace(']',"")\
    .replace('<br/>',"")\
    .replace('<br>',"")\
    .replace('</span>',"")\
    .replace('<span>',"")\
    .replace('.',"")\
    .replace(',',"")\
    .replace(';',"")\
    .lower()\
    .split()

#a token is a "word" made of two or more words like "data scientist"
tokens = words
for w in range(len(words)):
    if w != len(words)-1:
        w1 = words[w]
        w2 = words[w+1]
        w1_w2 = w1 + ' ' + w2
        tokens.append(w1_w2)
        if w!= len(words)-2:
            w1 = words[w]
            w2 = words[w+1]
            w3 = words[w+2]
            w1_w2_w3 = w1 + ' ' + w2 + ' ' + w3
            tokens.append(w1_w2_w3)
        else:
            pass
    else:
        pass

set(tokens)

{'em sistema de',
 'de frentes',
 'da área',
 'dos planos de',
 'planos de manutenção',
 'softwares específicos para',
 'montagem de banco',
 'passageiros',
 'python',
 'aproveitar ferramentas',
 'pessoas delegar',
 'e avaliar',
 'vale refeição vt',
 'na plataforma',
 'plr cursos',
 'trabalhe em colaboração',
 'técnica e economicamente',
 'industriais',
 'farmácia convênio com',
 'supervisionar as atividades',
 'sua satisfação',
 'de obra acompanhar',
 'a capacidade',
 'agente',
 'gestão dos indicadores',
 'airflow docker',
 'e partes',
 'privada plr -',
 'de projetos em',
 'api ou',
 'dos contratos analisar',
 'propondo melhorias',
 'conhecimento em leitura',
 'boa comunicação',
 'elétrica preferencialmente',
 'de comunicação',
 'em locais remotos',
 'resultados reembolso',
 'anos transporte',
 'projetos supervisionar as',
 'que os processos',
 'mantidos projetar e',
 'medidas para',
 'bem como proteger',
 'modelo de trabalho',
 '- programa de',
 'na cultura da',
 'e ou espanhol',
 'e

In [6]:
contagem_de_termos = []
for token in set(tokens):
    contagem_de_termos.append([token, text.count(token)])

In [7]:
stop_words = ['e','a', 'o', 'os', 'do', 'as', 'da', 'em', 'dos', 'se', 'no','com', 'na'\
        ,'para', 'ou', 'e a', 'é', '-', '/', 'de', 'um',' nos', 'que', 'das', 'por'\
        ,'como', '2', 'nos', 'off', 'e em', 'ao', 'com a', 'seu', 'são', 'etc', 'com o'\
        ,'esta', 'área', 'ano', 'dia', 'day','esta', 'nas']
df = pd.DataFrame(contagem_de_termos)
df[~(df[0].isin(stop_words)) & (df[1] >= 2) & (df[1] <= len(list_of_prequisites))].sort_values(by=1, ascending=False).reset_index(drop=True)

,0,1
0,temas,12
1,atua,12
2,trabalho,12
3,projeto,11
4,cad,10
5,ações,10
6,sistemas,10
7,segurança,10
8,dados e,9
9,spark,9


In [ ]:

#adicionar o título de vaga
#adicionar o termo da pesquisa
#associar o link da publicação com a descrição
#retirar tags
#contar o número de ocorrências de cada palavra
#armazenar o resultado
#rodar o programa duas vezes por dia e adicionar apenas novos registros
#adicionar a data a criação da vaga
#adicionar a data da consulta